In [146]:
import os.path

def doMergeDict(dict1, dict2):
    mergeDict = dict1
    for key in dict1.keys():
        if(not key in dict2.keys()):
            mergeDict[key] = dict1[key]
        else:
            mergeDict[key] = np.append(mergeDict[key], dict2[key])
    for key in dict2.keys():
        if(not key in dict1.keys()):
            mergeDict[key] = dict2[key]
    return mergeDict

class season:
    def calcDayPlayed(self):
        """ Calculate the played matchdays with help of the json-files.
        """
        matchday = 1
        while(os.path.exists(str(self.season)+'/'+str(matchday)+'.json')==True): 
            matchday +=1
        return matchday-1

    def loadTeams(self):
        ''' Return a array with the names of the teams which played in this season.
        '''
        data = leauge.loadMatchDay(self, self.season, 1)
        teams = []
        for x in range(len(data)):
            teams = np.append(teams, ([data[x]['Team1']['TeamName'], data[x]['Team2']['TeamName']]))
        return teams
    
    def loadScores(self, matchDay):
        data = leauge.loadMatchDay(self, self.season, matchDay)
            
        homeResults = dict()
        awayResults = dict()
        homeHalftime = dict()
        awayHalftime = dict()
    
        for x in range(len(data)):
            homeResults[data[x]['Team1']['TeamName']] = data[x]['MatchResults'][1]['PointsTeam1']
            awayResults[data[x]['Team2']['TeamName']] = data[x]['MatchResults'][1]['PointsTeam2']
            homeHalftime[data[x]['Team1']['TeamName']] = data[x]['MatchResults'][0]['PointsTeam1']
            awayHalftime[data[x]['Team2']['TeamName']] = data[x]['MatchResults'][0]['PointsTeam2']
        
        return dict(zip(['homeResults', 'awayResults', 'homeHalftime', 'awayHalftime'], [homeResults, awayResults, homeHalftime, awayHalftime]))
    
    def scoresSeason(self):
        matchScores = self.loadScores(1)
        matchScores = doMergeDict(matchScores, self.loadScores(2))
        return matchScores
    
    
    def loadPaarings(self, matchDay):
        '''
        Als numpy array weil man darin super nach Begegungen suchen kann ... Beispiel:
        a = np.array([[1,2],[3,4]])
        [1,2] == a
        array([[ True,  True],
               [False, False]], dtype=bool)
        '''
        data = leauge.loadMatchDay(self, self.season, matchDay)
        parings = np.array([[[data[0]['Team1']['TeamName']], [data[0]['Team2']['TeamName']]]])
        
        for x in range(len(data)-1):
            parings = np.append(parings, [[[data[x+1]['Team1']['TeamName']], [data[x+1]['Team2']['TeamName']]]], axis=0)
            
        return np.squeeze(parings)

    def paaringsSeason(self):
        matchPaarungen = [self.loadPaarings(1)]
        for x in range(self.currentMatchDay):
            matchPaarungen = np.append(matchPaarungen, [self.loadPaarings(x+1)], axis=0)
        return np.squeeze(matchPaarungen)
        
    def __init__(self, season):    
        self.season = season
        self.currentMatchDay = self.calcDayPlayed()   # Return the last completed matchday
        self.teams = self.loadTeams()                 # list of all teams which played in this season
        self.scores = self.scoresSeason()
        self.matchPaarungen = self.paaringsSeason()
            # Ein array an welchen Spieltach welche manschaft gegen Welche gespielt hatt.
    
        
exec(open('Einlesen.py').read())

In [150]:
season2016 = season(2016)
pprint(season2016.scoresSeason())
#season2016.loadScores(30)
#Begegnungen laden und nach einer bestimmten suchen
#a = season2016.loadPaarings(30)
#print(a)
#b = ['1. FC Köln','TSG 1899 Hoffenheim']
#b == a

{'awayHalftime': array([ {'SV Darmstadt 98': 0, 'VfL Wolfsburg': 1, 'FC Schalke 04': 0, '1. FSV Mainz 05': 1, 'Bayer 04 Leverkusen': 0, 'RB Leipzig': 0, 'SC Freiburg': 0, 'Werder Bremen': 0, 'FC Ingolstadt 04': 0},
       {'Eintracht Frankfurt': 0, 'Borussia Mönchengladbach': 1, 'Hamburger SV': 0, 'Hertha BSC': 1, 'TSG 1899 Hoffenheim': 1, '1. FC Köln': 0, 'Bayern München': 0, 'FC Augsburg': 0, 'Borussia Dortmund': 0}], dtype=object),
 'awayResults': array([ {'SV Darmstadt 98': 0, 'VfL Wolfsburg': 2, 'FC Schalke 04': 0, '1. FSV Mainz 05': 0, 'Bayer 04 Leverkusen': 1, 'RB Leipzig': 2, 'SC Freiburg': 1, 'Werder Bremen': 0, 'FC Ingolstadt 04': 1},
       {'Eintracht Frankfurt': 0, 'Borussia Mönchengladbach': 1, 'Hamburger SV': 1, 'Hertha BSC': 2, 'TSG 1899 Hoffenheim': 4, '1. FC Köln': 0, 'Bayern München': 2, 'FC Augsburg': 2, 'Borussia Dortmund': 0}], dtype=object),
 'homeHalftime': array([ {'Eintracht Frankfurt': 1, 'Borussia Mönchengladbach': 1, 'Hertha BSC': 0, 'Hamburger SV': 1, 'TSG

In [17]:
season2016.matchPaarungen == ['Bayern München']

array([[[ True, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False]],

       [[ True, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False]],

       [[False,  True],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False]],

       [[False, False],
        [False, False],
        [False, False],
        [False, False],
        [ True, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False]],

       [[False, False],
        [False, False],
        [False, False],
        [False, False],
        [False, False],
        

In [131]:
dict1 = {'team': 'bvb', 'players': 3}
dict1['blabla'] = 1
dict1['FUU'] = 1
dict2 = {'team': 'bvb', 'players': 3}
dict2['BAZZZ'] = 3
keys1 = dict1.keys()

def doMergeDict(dict1, dict2):
    mergeDict = dict1
    for key in dict1.keys():
        if(not key in dict2.keys()):
            mergeDict[key] = dict1[key]
        else:
            mergeDict[key] = np.append(mergeDict[key], dict2[key])
    for key in dict2.keys():
        if(not key in dict1.keys()):
            mergeDict[key] = dict2[key]
    return mergeDict

doMergeDict(dict1, dict2)

{'BAZZZ': 3,
 'FUU': 1,
 'blabla': 1,
 'players': array([3, 3]),
 'team': array(['bvb', 'bvb'], 
       dtype='<U3')}